In [ ]:
import os
import tensorflow as tf
import numpy as np
import keras
import pandas as pd
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [ ]:

def make_linear_model(D):

    input_layer = keras.layers.Input((D,))

    linear_layer = keras.layers.Dense(1)(input_layer)

    output_layer = keras.layers.Activation('sigmoid')(linear_layer)

    model = keras.models.Model(input_layer,output_layer)

    model.compile(loss='binary_crossentropy')

    return model




In [ ]:
# def make_linear_model(D):
#     # Define the input shape
#     input_layer = keras.layers.Input(shape=(D,))

#     # Add a linear (Dense) layer with 1 output neuron
#     linear_layer = keras.layers.Dense(1)(input_layer)

#     # Use sigmoid activation function for binary classification
#     output_layer = keras.layers.Activation('sigmoid')(linear_layer)

#     # Create the model
#     model = keras.models.Model(inputs=input_layer, outputs=output_layer)

#     # Compile the model with binary crossentropy loss and an optimizer
#     model.compile(optimizer='adam', loss='binary_crossentropy')

#     return model

In [ ]:
D = 10  # Number of features
model = make_linear_model(D)

In [ ]:
D = 10  # Number of features
model_ava1 = make_linear_model(D)

In [ ]:
#example psuedo data

# inputs = np.random.random((1000, D))  # 1000 samples, D features each
# labels = np.random.randint(0, 2, (1000, 1))  # 1000 binary labels

In [ ]:
#actual data import
stimulus = np.load('/Users/iuliarusu/Documents/Sharpee/ProcAiryData/inpfull_op50_SF.npz')
Y_full = np.load('/Users/iuliarusu/Documents/Sharpee/ProcAiryData/Yfull_op50_SF.npz')

In [ ]:
#import AVA data from all cells
AVA_0_df = pd.read_csv ('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_0_df.csv')
AVA_1_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_1_df.csv') 
AVA_2_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_2_df.csv')
AVA_3_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_3_df.csv') 
AVA_4_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_4_df.csv') 
AVA_5_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_5_df.csv') 
AVA_6_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_6_df.csv')

In [ ]:
AVA_0 = np.array(AVA_0_df.iloc[:, 1: -2])

In [ ]:
AVA_0.shape

In [ ]:
stimulus_concat = np.concatenate([stimulus['arr_0'], stimulus['arr_1'], stimulus['arr_2'], stimulus['arr_3'], stimulus['arr_4'], stimulus['arr_5'], stimulus['arr_6']], axis =1)

In [ ]:
# stimulus_concat = np.concatenate([stimulus['arr_0'], stimulus['arr_1'], stimulus['arr_2'], stimulus['arr_3'], stimulus['arr_4'], stimulus['arr_5'], stimulus['arr_6']], axis =1)
# Y_full_concat = np.concatenate([Y_full['arr_0'],
#                          Y_full['arr_1'],
#                          Y_full['arr_2'],
#                          Y_full['arr_3'] , Y_full['arr_4'], Y_full['arr_5'], Y_full['arr_6']], axis =1)

In [ ]:

# def get_bins(stimulus, response, bin_size, start_idx, cell_idx, num_bins = 10):
#     response_vector = []
#     stimulus_vector = []
#     stimulus_matrix = ((num_bins, bin_size))

#     if start_idx < num_bins * bin_size:
#         print("Start index is out of bounds.")
#         return None
#     start = response[cell_idx,  start_idx: start_idx + 10] 
#     response_vector.append(start)
#     end = start_idx - (bin_size * num_bins)
#     print('end', end)
#     print("resp", response_vector)
#     stimulus_vector.append(stimulus[ 0, end : start_idx ])
#     print("stim", stimulus_vector)
#     stimulus_vector = np.array(stimulus_vector)
#     stimulus_matrix = stimulus_vector.reshape((num_bins, bin_size))
#     return stimulus_matrix, np.array(response_vector)

In [ ]:
def get_bins_avg(stimulus, response, bin_size, start_idx, cell_idx, num_bins=10, nsamples=10):
    response_vector = []
    stimulus_vector = []
    for _ in range(nsamples):
        r = response[cell_idx, start_idx: start_idx + bin_size]
        response_vector.append(np.average(r))
        print('response', response_vector)

        bin_stimuli = []
        for i in range(num_bins):
           
            end_idx = (start_idx ) - (i * bin_size)
            print('start_idx', start_idx)
            print('end', end_idx)
            start_idx_bin = end_idx + bin_size
            print('start_idx_bin', start_idx_bin)

            # Check if the calculated index is within the bounds
            if start_idx_bin < 0:
                break

            # Append the average of stimulus over the bin
            bin_stimuli.append(np.average(stimulus[0, end_idx:start_idx_bin]))
            print('stim', stimulus[0, start_idx_bin:end_idx])
            print('bin_stimuli', bin_stimuli)

        # Collect all averages per sample
        stimulus_vector.append(bin_stimuli)
        print('stimulus_vector', stimulus_vector)

        # Update start index for the next sample
        start_idx += bin_size

    # Convert list of lists into a 2D NumPy array
    stimulus_matrix = np.array(stimulus_vector)  # Transpose to match expected dimensions

    return stimulus_matrix, np.array(response_vector)


In [ ]:
AVA_0[0, 200: 200 + 10]

for a single cell:

In [ ]:


bin_size = 10
start_idx = 200  # Starting index for the stimulus and response extraction
num_bins = 10
nsamples = 100 #each time point gets multiplied by bin size, as you go along, if you start at t= 200, bin size 10, you'll end at time point 200 + 1000(10) is nsamples = 1000

stimulus = stimulus_concat.T
response = AVA_0
cell_idx = 0 #cell in response

data, label = get_bins_avg(stimulus, response, bin_size, start_idx, cell_idx, num_bins, nsamples)




Loop over each cell in the dataframe

In [ ]:
#Loop over each cell index in the DataFrame

for cell_idx in range(response.shape[0]//2):
    bin_size = 5
    start_idx = 200  # Starting index for the stimulus and response extraction
    num_bins = 10
    nsamples = 100 #amount of times you're moving over the data

    stimulus = stimulus_concat.T
    response = AVA_1
    cell_idx = 0 #cell in response

    data, label = get_bins_avg(stimulus, response, bin_size, start_idx, cell_idx, num_bins, nsamples)

In [ ]:
data.shape

In [ ]:
label.shape

In [ ]:
AVA_1 = np.array(AVA_1_df.iloc[:, 1: -2])

In [ ]:

bin_size = 10
start_idx = 200  # Starting index for the stimulus and response extraction
num_bins = 10
nsamples = 100 #each time point gets multiplied by bin size, as you go along, if you start at t= 200, bin size 10, you'll end at time point 200 + 1000(10) is nsamples = 1000

stimulus = stimulus_concat.T
response = AVA_1
cell_idx = 0 #cell in response

data, label = get_bins_avg(stimulus, response, bin_size, start_idx, cell_idx, num_bins, nsamples)

In [ ]:
# Initialize containers for data and labels
all_data = []
all_labels = []

bin_size = 10
start_idx = 200  # Starting index for the stimulus and response extraction
num_bins = 10
nsamples = 100 #amount of times you're moving over the data

stimulus = stimulus_concat.T
response = AVA_1
cell_idx = 0 #cell in response
# Calculate data and labels for each cell or half the cells
num_cells = response.shape[0]//2  # or response.shape[0] // 2 for half of the cells
for cell_idx in range(num_cells):
    data, label = get_bins_avg(stimulus, response, bin_size, start_idx, cell_idx, num_bins, nsamples)
    all_data.append(data)
    all_labels.append(label)

In [ ]:
all_data_array = np.concatenate(all_data, axis=0)

# Convert list of 1D arrays to a single 2D array (if all_labels consists of 1D arrays)
all_labels_array = np.concatenate(all_labels, axis=0)

In [ ]:
all_data_array.shape

In [ ]:
all_labels_array.shape

In [ ]:
inputs = data
labels = label

In [ ]:

inputs = all_data_array  # 100 samples, D features each
labels = all_labels_array  # 100 labels




In [ ]:
#train
model.fit(inputs, labels, epochs=10, batch_size=32)

In [ ]:
model_ava1.fit(inputs, labels, epochs=10, batch_size=32)

In [ ]:
bin_size = 10
start_idx = 200  # Starting index for the stimulus and response extraction
num_bins = 10
nsamples = 50

stimulus = stimulus_concat.T
response = AVA_0
cell_idx =  9 #cell in response

data_test, label_test = get_bins_avg(stimulus, response, bin_size, start_idx, cell_idx, num_bins, nsamples)

In [ ]:
X_test = data_test  # your test stimuli

# Predict the response probabilities
predicted_probabilities = model.predict(X_test)

# If you need binary responses (firing or not firing), you can threshold these probabilities:
predicted_responses = (predicted_probabilities >= 0.5).astype(int)

In [ ]:
model.weights

model.weights[0]



In [ ]:
model_ava1.weights

model_ava1.weights[0]

In [ ]:
fig, ax = plt.subplots()
ax.plot(model_ava1.weights[0])
plt.title("Linear MNE: AVA Receptive Field for single AVA cell in Worm 0 ")

In [ ]:
fig, ax = plt.subplots()
ax.plot(model.weights[0])
plt.title("Linear MNE: AVA Receptive Field for AVA all cell in Worm 0 ")

In [ ]:
fig, ax = plt.subplots()
ax.plot(model.weights[0])
plt.title("Linear MNE: AVA Receptive Field for AVA cell in Worm 0 ")

In [ ]:
#bulk processing of all cells in a 